# Jupyter Noteboook OpenCV Webcam Test

In [3]:
import cv2
import time
from IPython.display import display, update_display, clear_output, Image
import ipywidgets as widgets
from io import BytesIO
import PIL.Image
import numpy as np
from copy import copy
import threading
import bqplot as bq

if 'cam' in vars():
    cam.release()

cam = cv2.VideoCapture(0)
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# face_detector = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')
# profile_detector = cv2.CascadeClassifier('haarcascade_profileface.xml')
face_landmark = cv2.face.createFacemarkLBF()
face_landmark.loadModel('lbfmodel.yaml')

def draw_hist(frame, to_draw, bin_width=1, bar_width=1, top=10, left=10, height=100):
    global histogram, bar
    r = frame[:,:,2].flatten()
    g = frame[:,:,1].flatten()
    b = frame[:,:,0].flatten()
    
    r_hist, _ = np.histogram(r, bins=list(range(0, 256, bin_width)), range=(0, 255))
    g_hist, _ = np.histogram(g, bins=list(range(0, 256, bin_width)), range=(0, 255))
    b_hist, _ = np.histogram(b, bins=list(range(0, 256, bin_width)), range=(0, 255))
    histogram = np.c_[r_hist, g_hist, b_hist]
    histogram = histogram / histogram.max()
    
    width = (bar_width) * len(histogram) + 1
    for i, bar in enumerate(histogram):
        x = left + 1 + i * (bar_width)
#         y = top + int(height * (1 - histogram[i]))
#         cv2.rectangle(frame, (x, y), (x+bar_width, top+height),
#                       (0, 0, 255), -1)
        il = np.argmin(bar)
        ih = np.argmax(bar)
        if il == 0:
            im = 2 if ih == 1 else 1
        elif il == 1:
            im = 2 if ih == 0 else 0
        else:
            im = 1 if ih == 0 else 0
        yl = top + int(height * (1 - bar[il]))
        cl = [255, 255, 255]
        ym = top + int(height * (1 - bar[im]))
        cm = [255, 255, 255]
        cm[2-il] = 0
        yh = top + int(height * (1 - bar[ih]))
        ch = [255, 255, 255]
        ch[2-il] = 0
        ch[2-im] = 0
        cv2.rectangle(to_draw, (x, yl), (x+bar_width+1, top+height), cl, -1)
        cv2.rectangle(to_draw, (x, ym), (x+bar_width+1, yl), cm, -1)
        cv2.rectangle(to_draw, (x, yh), (x+bar_width+1, ym), ch, -1)
    cv2.rectangle(to_draw, (left, top), (left+width, top+height),
                  (255, 255, 255), bar_width // 2)
    
    return frame

def get_frame():
    _, frame = cam.read()
    annotated = frame.copy()
    
    # Detect faces locations
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    
    faces = face_detector.detectMultiScale(gray, 1.2, 5)
#     profiles = profile_detector.detectMultiScale(gray, 1.2, 5)

    face = None
    
    for i, (x, y, w, h) in enumerate(faces):
        cv2.rectangle(annotated, (x, y), (x+w, y+h), (0, 255, 0), 2)
        if i == 0:
            face = frame[y:y+h, x:x+w]
    
#     for (x, y, w, h) in profiles:
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    if len(faces) > 0:
        _, landmarks = face_landmark.fit(gray, faces)

        for landmark in landmarks:
            for x, y in landmark[0].astype(np.int):
                cv2.circle(annotated, (x, y), 1, (0, 0, 255), 2)
                
    draw_hist(frame, annotated)
    
    return face, annotated

def get_bytestring(frame):    
    # Output Byte String
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    f = BytesIO()
    PIL.Image.fromarray(frame).save(f, 'jpeg')
    return f.getvalue()

last_time = time.time()
ts = np.random.rand(150)
_, annotated = get_frame()

im = widgets.Image(value=get_bytestring(annotated),
                   layout=widgets.Layout(width='640px', height='480px'))
button = widgets.ToggleButton(value=False, description=' Play', 
                              tooltip='Play the webcam stream', icon='play')
fps_box = widgets.Label(value='FPS: ')

xsc = bq.LinearScale()
ysc = bq.LinearScale()

l = bq.Lines(x=np.arange(len(ts)), y=ts, scales={'x': xsc, 'y': ysc})

xax = bq.Axis(scale=xsc)
yax = bq.Axis(scale=ysc, orientation='vertical')

f = bq.Figure(marks=[l], axes=[xax, yax], 
              layout=widgets.Layout(width='640px', height='480px'))

def thread_function():
    global im, button, fps_box, l, last_time, ts
    while button.value:
        face, annotated = get_frame()
        ts = np.roll(ts, -1)
        if face is not None:
            ts[-1] = face[:,:,1].mean()
        else:
            ts[-1] = ts[-2]
        l.y = ts
        im.value = get_bytestring(annotated)
        fps_box.value = 'FPS: %5.2f' % (1 / (time.time() - last_time))
        last_time = time.time()

def button_callback(change):
    if change['new']:
        button.description = ' Playing'
        thread = threading.Thread(target=thread_function)
        thread.start()
    else:
        button.description = ' Play'

button.observe(button_callback, 'value')

display(widgets.VBox([widgets.HBox([button, fps_box]), widgets.HBox([im, f])]))

0.0